# 2024-1 DSL 정규세션 과제

## CNN Assignment

- 작성자 : 윤형진
- 참고자료 : PyTorch Document
- 과제 제출 기한 : 02.18

---
- 이름 :     
- 기수 :     



In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [2]:
# colab 환경에서 학습을 진행하실 분들은 구글드라이브를 연동해주세요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 이번 과제의 일부였던 CustomDataset.py, Model.py를 로드해주세요
#from CustomDataset import CustomImageDataset
#from Model import ResNet50

import pandas as pd
import numpy as np

import yaml
import os

import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [4]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

from PIL import Image

"""
MetaData.csv를 나누어 만든 train, val, test 데이터프레임에 담긴 이미지와 레이블을 반환해주는 CustomImageDataset을 만들어주세요
참고자료 : https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
"""

class CustomImageDataset(Dataset):

    def __init__(self, dataset_direc:str, df:pd.DataFrame, transform = None) -> None:

        # 3000개의 이미지를 담고있는 DataSet 디렉토리의 경로입니다.
        self.dataset_direc= dataset_direc

        self.df = df.reset_index(drop=True)

        self.to_tensor = ToTensor() # 파이토치에서는 모든 인풋을 torch.Tensor()로 바꿔줘야합니다.

        self.transform = transform

    # __init__() 함수에서 받은 데이터프레임에 포함된 전체 데이터포인트의 수를 반환하도록 해주세요
    def __len__(self) -> int :
        return len(self.df)



    def __getitem__(self, idx:int):

        final_directory = os.path.join(self.dataset_direc, self.df['StrLabel'].iloc[idx], self.df['FileName'].iloc[idx])

        image = Image.open(final_directory)

        label = self.df['Label'].iloc[idx]

        image = self.to_tensor(image)

        if self.transform:
            image = self.transform(image)

        return image, label

class ResBottleNeck(nn.Module):
    expansion = 4  # Expansion value for bottleneck

    def __init__(self, in_dim, out_dim, stride=1) -> None:
        super(ResBottleNeck, self).__init__()
        dim = out_dim * ResBottleNeck.expansion
        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels=in_dim, out_channels=out_dim, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=out_dim, out_channels=out_dim, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=out_dim, out_channels=dim, kernel_size=1, bias=False),
            nn.BatchNorm2d(dim)
        )

        self.relu = nn.ReLU(inplace=True)
        self.downsample = None
        if stride != 1 or in_dim != dim:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels=in_dim, out_channels=dim, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(dim)
            )

    def forward(self, x):
        residual = x
        out = self.bottleneck(x)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


class ResNet50(nn.Module):
    def __init__(self, num_classes=1) -> None:
        super(ResNet50, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 64, blocks=3)
        self.layer2 = self._make_layer(256, 128, blocks=4, stride=2)
        self.layer3 = self._make_layer(512, 256, blocks=6, stride=2)
        self.layer4 = self._make_layer(1024, 512, blocks=3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)

    def _make_layer(self, in_dim, out_dim, blocks, stride=1):
        layers = []
        layers.append(ResBottleNeck(in_dim, out_dim, stride))
        for _ in range(1, blocks):
            layers.append(ResBottleNeck(out_dim * ResBottleNeck.expansion, out_dim))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
# config.yaml의 위치를 지정해주세요
config_path = "<여러분의 디렉토리>/scripts/config.yaml"

# 1. 하이퍼파라미터 및 기타 주요 변수 설정

config.yaml에 지정해줬던 디렉토리, 모델 하이퍼파라미터들을 가져옵시다

In [5]:

dataset_direc = "/content/drive/MyDrive/[240206]_CNN_YOURNAME/DataSet"
meta_direc = "/content/drive/MyDrive/[240206]_CNN_YOURNAME/scripts/MetaData.csv"
weight_direc  = "/content/drive/MyDrive/[240206]_CNN_YOURNAME/BestModel"


epoch, batch_size, lr = 30, 80, 0.001


In [7]:
import wandb # wandb.ai으로의 실시간 연동을 위한 라이브러리 임포트
from signal import signal, SIGPIPE, SIG_DFL
#Ignore SIG_PIPE and don't throw exceptions on it... (http://docs.python.org/library/signal.html)
signal(SIGPIPE,SIG_DFL)

# wandb login
wandb.init(
    project="DSL_Noarg", entity="woni5806",
    name="Tumor no arg",
    config={
        "model_name": "no arg",
        "epochs": epoch,
        "batch_size": batch_size,
        "lr": lr
    })

config = wandb.config
#42f2dd32632680997d4be527bcfc8a91bb7265f0
#42f2dd32632680997d4be527bcfc8a91bb7265f0

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
# 가속을 위해 device로 GPU로 설정해줍니다
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# 2. train, val, test DataLoader 초기화

In [9]:

# MetaData.csv를 로드해주고, train, val, test로 나눠줍니다
meta_data = pd.read_csv(meta_direc)

# train : val : test = 6 : 2 : 2
train, val = train_test_split(meta_data, train_size=0.6, random_state=777)
val_test, test = train_test_split(meta_data, train_size=0.5, random_state=777)

# Model generalization을 위해 RandomAugmentation에 어떤 변형을 가할 지 transforms.Compose([])로 묶어놓읍시다


# CustomDataset.py에서 가져온 CustomImageDataset 클래스에 train, val, test 데이터프레임을 패스하여 데이터셋을 만들어주세요
train_dataset = CustomImageDataset(dataset_direc=dataset_direc, df=meta_data , transform = None)
val_dataset   = CustomImageDataset(dataset_direc=dataset_direc, df=meta_data , transform = None)
test_dataset  = CustomImageDataset(dataset_direc=dataset_direc, df=meta_data , transform = None)


train_loader = DataLoader(train_dataset , batch_size = 64 ,shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size = 64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size = 64, shuffle=True)

# 3. 모델 초기화, 손실함수 및 optimizer 지정

In [10]:
# model을 선언하고 GPU에 올려줍니다
model = ResNet50()
model.to(device)
# 손실함수를 정의해주세요
criterion =  nn.BCEWithLogitsLoss()
# optimizer를 정의해주세요
optimizer = optim.Adam(model.parameters(), lr=lr)
# 손실함수를 정의해주세요


# 4. Training 시작

학습을 위한 모든 준비가 끝났습니다! ㄱㅂㅈㄱ

In [11]:
# Best Validation Loss를 양의 무한대로 초기화
best_loss = float("inf")


# Training, Validation 시작

for curr_epoch in range(epoch+1): # iterate over all EPOCHS
  train_loss = 0
  correct_train = 0

  for img, label in (train_loader): # iterate over all BATCHES

    # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
    img, label = img.to(device), label.to(device)



    # Forward Propagation
    pred = model(img)
    label = label.view(-1,1).float()

    # 손실함수 계산
    loss = criterion(pred, label) # 구현상의 차이로 label 텐서에 조작을 가해 모양을 맞춰줘야하는 경우가 생길 수 있습니다 ( 힌트 : tensor.view() )

    # 누적된 기울기 초기화 및 역전파
    optimizer.zero_grad()
    loss.backward()

    # 파라미터 업데이트
    optimizer.step()

    # 해당 Batch에서의 loss를 train_loss 변수에 누적
    train_loss += loss.item()

    # Accuracy 계산
    pred_class = (torch.sigmoid(pred) > 0.5).float()
    correct_train += (pred_class == label).float().sum()

  # 모든 Batch를 순회한 이후 최종적인 train_loss, train_acc를 계산해줍니다
  train_loss = train_loss/len(train_loader.dataset)
  train_acc = correct_train /len(train_loader.dataset)

  # Validation 시작
  model.eval()

  with torch.no_grad():  # 학습 중이 아니므로 역전파를 위한 gradient 연산은 필요하지 않습니다
    val_loss = 0
    correct_val = 0

    for img, label in (val_loader): # iterate over all BATCHES
        # GPU에 올라가는 건 모델 뿐 아니라 데이터도 같이 올라가야합니다!
      img, label = img.to(device), label.to(device)
      label = label.view(-1,1).float()

        # Forward Propagation
      pred = model(img)

        # 손실함수 계산
      loss = criterion(pred, label)

      val_loss += loss.item()

      # 모든 Batch를 순회한 이후 최종적인 val_loss, val_acc를 계산해줍니다
      pred_class = (torch.sigmoid(pred) > 0.5).float()
      correct_val += (pred_class == label).float().sum()

    val_loss /= len(val_loader)
    val_acc = correct_val / len(val_loader)

  # wandb.ai에 보낼 log 데이터를 작성해줍니다.
  metrics = {
    "epoch": epoch,
    "train_loss": train_loss,
    "val_loss" : val_loss,
    "train_acc": train_acc.item(),
    "val_acc" : val_acc.item(),
    "learning_rate": lr
    }

  # 아래 코드를 통해 실시간으로 학습현황을 확인할 수 있도록 log 데이터를 전송해줍니다
  wandb.log(metrics)

  # 모델을 다시 학습모드로 설정합니다
  model.train()

  # 가장 작은 validation loss를 가졌을 때의 모델 파라미터를 저장해줍니다.
  if val_loss < best_loss:
    best_loss = val_loss
    torch.save(model.state_dict(), os.path.join(weight_direc, "best_model(aug).pt")) # .pt 확장자로 저장해주도록합니다

wandb.finish()
